### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### GOOGLE SHEETS

In [100]:
# Ciudades Argentina
sheet_id = '1DZfQAfKlt-9MrTBQ65NhVSMPW8CQd1An_teq-RjQRtM'
wks_name = 'Argentina'
sheet = Spread(sheet_id, wks_name, config=cred)
city_arg = sheet.sheet_to_df(index=0,header_rows=1)

In [101]:
# Ciudades Chile
sheet_id = '1DZfQAfKlt-9MrTBQ65NhVSMPW8CQd1An_teq-RjQRtM'
wks_name = 'Chile'
sheet = Spread(sheet_id, wks_name, config=cred)
city_ch = sheet.sheet_to_df(index=0,header_rows=1)

In [102]:
# Concateno ambas listas
competition = pd.concat([city_arg,city_ch],ignore_index=True)
# Doy formato a las columnas
competition['City_Final'] = competition['City_Final'].str.upper()

### CONSTANTES

In [91]:
inicio = '2021-04-01'

### QUERIES

In [118]:
q_orders = '''SELECT cn.country_name AS Country,
       IFNULL(c.city_name,'-') AS City,
       IFNULL(a.area_name,'-') AS Area,
       CASE WHEN c.city_name = 'Buenos Aires' THEN IFNULL(a.area_name,'-') ELSE IFNULL(c.city_name,'-') END AS City_Final,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       IFNULL(dt.discount_type_name,'-') AS Discount_Type,
       IFNULL(od.discount_paid_by,'-') AS Paid_By,
       COUNT(DISTINCT o.order_id) AS Confirmed,
       ROUND(IFNULL(SUM(od.discount_amount / ce.rate_us),0),2) AS Amount
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
UNNEST (discounts) AS od
LEFT JOIN `peya-bi-tools-pro.il_core.dim_discount_type` AS dt ON od.discount_type_id = dt.discount_type_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,month) = ce.currency_exchange_date
WHERE o.registered_date >= DATE('{0}')
      AND o.order_status = 'CONFIRMED'
      AND cn.country_name IN ('Argentina','Chile')
      AND dt.discount_type_name IS NOT NULL
GROUP BY 1,2,3,4,5,6,7
ORDER BY 1,2,3,4,5,6,7 DESC'''.format(inicio)

In [119]:
# Descargo la data
bq_orders = pd.io.gbq.read_gbq(q_orders, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 31367/31367 [00:04<00:00, 7430.57rows/s]


In [134]:
# Copio las bases
orders = bq_orders.copy()

### TRABAJO

In [135]:
# Doy formato a las columnas
strings = ['Country','City','Area','City_Final','Month','Discount_Type','Paid_By']
cols = [i for i in orders.columns if i not in strings]
orders[cols] = orders[cols].astype(float)

In [136]:
# Mando City a mayusculas para el merge
orders['City'] = orders['City'].str.upper()
orders['Area'] = orders['Area'].str.upper()
orders['City_Final'] = orders['City_Final'].str.upper()
# Coloco el nivel de competicion
orders = orders.merge(competition,on=['City_Final'],how='left')
orders.replace([np.nan,np.inf,-np.inf],'-',inplace=True)
# Creo la columna de cruce
orders['Cruce'] = orders['City']+'-'+orders['Area']

In [138]:
# Creo un crudo general por pais
index = ['Month','Country','Discount_Type','Paid_By']
values = ['Confirmed','Amount']
country = orders.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

In [139]:
# Ordeno las tablas
orders.sort_values(by=['Month','Country','City','Area','City_Final'],ascending=True,inplace=True)
country.sort_values(by=['Month','Country'],ascending=True,inplace=True)

### CARGA

In [130]:
# Carga Crudo
sheet_id = '1GCdaSnPWJj7nrBruw62tNTVrmJ6JPLy2AxGrCdp1Vd0'
wks_name = 'Crudo Country'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(country, index=False, sheet=wks_name, replace=True)

In [140]:
# Carga Crudo Ciudades
sheet_id = '1GCdaSnPWJj7nrBruw62tNTVrmJ6JPLy2AxGrCdp1Vd0'
wks_name = 'Crudo Ciudades'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(orders, index=False, sheet=wks_name, replace=True)